<a href="https://colab.research.google.com/github/Rishita300/rishita/blob/master/Copy_of_DEC25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import cv2
import numpy as np
import os

np.random.seed(600)



class DriveFile:
    def __init__(self, base_path=""):
        self.path = base_path

    def dataload(self, folder_path):
        img = []
        label = []
        self.path1 = os.path.join(self.path, folder_path)

        if os.path.exists(self.path1):
            for i in os.listdir(self.path1):
                self.dis_path = os.path.join(self.path1, i)

                # Skip invalid directories
                if not os.path.exists(self.dis_path) or '.DS_Store' in self.dis_path:
                    print(f"Skipping invalid path or .DS_Store: {self.dis_path}")
                    continue

                for j in os.listdir(self.dis_path):
                    if j.endswith(('.jpg', '.png', '.jpeg')):
                        self.img_path = os.path.join(self.dis_path, j)
                        loaded_img = self.imgload(self.img_path)
                        if loaded_img is not None:
                            img.append(loaded_img)
                            label.append(i)
                        else:
                            print(f"Skipping non-image file or invalid image: {j}")
            if img and label:
                return np.array(img), np.array(label)
            else:
                print("No valid images found in the specified path.")
                return None, None
        else:
            print("Base path not found:", self.path1)
            return "patherror", "patherror"

    def imgload(self, imgpath):
        try:
            img = cv2.imread(imgpath)
            if img is None:
                print(f"Failed to load image at {imgpath}")
                return None
            img = cv2.resize(img, (20, 20))  # Downsample to 20x20 (or change size if needed)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
            return np.array(img)
        except Exception as e:
            print(f"Error processing image {imgpath}: {e}")
            return None

In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
drive = DriveFile()

# Load training data
images, labels = drive.dataload('/content/drive/MyDrive/train_set')
if images is not None and labels is not None:
    images = (images - np.mean(images)) / np.std(images)  # Normalize training images
else:
    print("Failed to load training data.")

# Load testing data
test_images, test_labels = drive.dataload('/content/drive/MyDrive/test_set')
if test_images is not None and test_labels is not None:
    test_images = (test_images - np.mean(test_images)) / np.std(test_images)  # Normalize testing images
else:
    print("Failed to load testing data.")

In [120]:
import numpy as np

def encode_labels(labels, label_to_index):
    return np.array([label_to_index[label] for label in labels])

def one_hot_encode(labels, unique_labels):
    return np.eye(len(unique_labels))[labels]

# Flattening images
images = np.array([img.flatten() for img in images])
test_images = np.array([img.flatten() for img in test_images])

# Label encoding
labels = np.array(labels)
test_labels = np.array(test_labels)

# Generate label-to-index mapping
unique_labels = np.unique(labels)
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

# Integer encoding
integer_encoded = encode_labels(labels, label_to_index)
test_integer_encoded = encode_labels(test_labels, label_to_index)

# One-hot encoding
one_hot_encoded = one_hot_encode(integer_encoded, unique_labels)
test_one_hot_encoded = one_hot_encode(test_integer_encoded, unique_labels)

# Assign one-hot encoded labels back
labels = one_hot_encoded
test_labels = test_one_hot_encoded

In [121]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Convert one-hot encoded labels back to integer labels
labels_int = np.argmax(labels, axis=1)
test_labels_int = np.argmax(test_labels, axis=1)

# Standardize the images
scaler = StandardScaler()
images_scaled = scaler.fit_transform(images)
test_images_scaled = scaler.transform(test_images)

# Hyperparameter tuning using GridSearchCV
parameters = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(random_state=42), parameters, cv=5)
grid_search.fit(images_scaled, labels_int)
print("Best Parameters:", grid_search.best_params_)

# Initialize and train SVM model with best parameters
svm_model = grid_search.best_estimator_
svm_model.fit(images_scaled, labels_int)

# Make predictions and evaluate the model
svm_predictions = svm_model.predict(test_images_scaled)
svm_accuracy = accuracy_score(test_labels_int, svm_predictions)

# Print accuracy and classification report
print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(test_labels_int, svm_predictions))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_labels_int, svm_predictions))

Best Parameters: {'C': 10, 'kernel': 'rbf'}
SVM Accuracy: 77.68%
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.67      0.66        33
           1       0.70      0.80      0.74        20
           2       0.92      0.75      0.83        32
           3       0.61      0.67      0.64        33
           4       0.86      0.78      0.82        23
           5       0.70      0.76      0.73        25
           6       0.97      0.97      0.97        34
           7       0.84      0.82      0.83        33

    accuracy                           0.78       233
   macro avg       0.78      0.78      0.78       233
weighted avg       0.79      0.78      0.78       233

Confusion Matrix:
[[22  0  1  5  2  2  0  1]
 [ 0 16  0  3  0  1  0  0]
 [ 1  3 24  2  0  1  0  1]
 [ 6  1  1 22  0  1  1  1]
 [ 1  1  0  0 18  2  0  1]
 [ 3  0  0  2  0 19  0  1]
 [ 0  1  0  0  0  0 33  0]
 [ 1  1  0  2  1  1  0 27]]


In [122]:
class Layer:
    def __init__(self):
        self.input=None
        self.output=None
    def forward(self,input):
        # to be overridden
        pass
    def backward(self,output_gradient,learning_rate):
        pass

In [123]:


class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)
        self.bias = np.zeros((1, output_size))
        self.velocities_W = np.zeros_like(self.weights)
        self.velocities_B = np.zeros_like(self.bias)

    def forward(self, input):
        self.input = input
        return np.dot(self.input, self.weights) + self.bias

    def backward(self, output_gradient, learning_rate, momentcoeff):
        # Compute gradients
        w_gradient = np.dot(self.input.T, output_gradient)  # Gradient w.r.t weights
        b_gradient = np.sum(output_gradient, axis=0, keepdims=True)  # Gradient w.r.t bias
        input_gradient = np.dot(output_gradient, self.weights.T)  # Gradient w.r.t input

        # Apply momentum
        self.velocities_W = momentcoeff * self.velocities_W + (1 - momentcoeff) * w_gradient
        self.velocities_B = momentcoeff * self.velocities_B + (1 - momentcoeff) * b_gradient

        # Update weights and biases with momentum
        self.weights -= learning_rate * self.velocities_W
        self.bias -= learning_rate * self.velocities_B

        # Optionally apply gradient clipping
        # max_norm = 5.0
        # if np.linalg.norm(w_gradient) > max_norm:
        #     w_gradient = w_gradient * (max_norm / np.linalg.norm(w_gradient))

        return input_gradient

In [124]:
class Activation(Layer):
    def __init__(self, activation, activation_derivative):
        # 'activation' is a function pointer to the activation method (e.g., sigmoid, relu)
        self.activation = activation
        self.activation_derivative = activation_derivative

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate, momentcoeff):
        # Compute the gradient of the loss with respect to the input
        activation_gradient = self.activation_derivative(self.input)
        # Return the product of the output gradient and the activation gradient
        return output_gradient * activation_gradient

In [125]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

class Sigmoid(Activation):
    def __init__(self):
        # Call the parent constructor with the sigmoid activation function and its derivative
        super().__init__(activation=sigmoid, activation_derivative=sigmoid_derivative)

In [126]:
def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

def tanh_derivative(x):
    t = tanh(x)
    return 1 - (t ** 2)

class Tanh(Activation):
    def __init__(self):
        # Call the parent constructor with the tanh activation function and its derivative
        super().__init__(activation=tanh, activation_derivative=tanh_derivative)

In [127]:
def prelu(x, alpha=0.1):
    return np.where(x > 0, x, alpha * x)

def prelu_derivative(x, alpha=0.1):
    return np.where(x > 0, 1, alpha)

class Relu(Activation):
    def __init__(self, alpha=0.1):
        # Store alpha for use in PReLU
        self.alpha = alpha
        # Pass the prelu and prelu_derivative functions to the parent class
        super().__init__(activation=lambda x: prelu(x, self.alpha), activation_derivative=lambda x: prelu_derivative(x, self.alpha))

In [128]:
class Softmax:
    def __init__(self, input_size, output_size):
        # Initialize weights and biases
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)
        self.bias = np.zeros((1, output_size))
        self.velocities_W = np.zeros_like(self.weights)
        self.velocities_B = np.zeros_like(self.bias)

    def forward(self, input):
        # Save input for backward pass
        self.input = input
        # Compute logits (raw scores)
        logits = np.dot(self.input, self.weights) + self.bias
        # Numerically stable softmax computation
        logits_shifted = logits - np.max(logits, axis=1, keepdims=True)  # For numerical stability
        exp_values = np.exp(logits_shifted)
        # Softmax output (probabilities)
        self.softmax = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        return self.softmax

    def backward(self, output_gradient, learning_rate, momentcoeff):
        # Gradient of softmax combined with cross-entropy loss
        # This simplifies to the difference between the predicted and actual labels
        batch_size = self.input.shape[0]

        # Compute the gradients for weights, biases, and inputs
        # output_gradient is typically the gradient of the loss with respect to the output of this layer
        w_gradient = np.dot(self.input.T, output_gradient) / batch_size
        b_gradient = np.sum(output_gradient, axis=0, keepdims=True) / batch_size
        input_gradient = np.dot(output_gradient, self.weights.T)

        # Update weights and biases with momentum
        self.velocities_W = momentcoeff * self.velocities_W + (1 - momentcoeff) * w_gradient
        self.velocities_B = momentcoeff * self.velocities_B + (1 - momentcoeff) * b_gradient
        self.weights -= learning_rate * self.velocities_W
        self.bias -= learning_rate * self.velocities_B

        return input_gradient

In [129]:
class Loss(Layer):
    def __init__(self, loss_fn, loss_fn_derivative):
        # Loss function and its derivative are passed in the constructor
        self.loss_fn = loss_fn
        self.loss_fn_derivative = loss_fn_derivative

    def forward(self, y_pred, y_true):
        # Store predictions and true values for backward pass
        self.y_pred = y_pred
        self.y_true = y_true
        # Return the computed loss value
        return self.loss_fn(y_pred, y_true)

    def backward(self):
        # Return the gradient of the loss with respect to the predictions
        return self.loss_fn_derivative(self.y_pred, self.y_true)

In [130]:
class BinaryCrossEntropy(Loss):
    def __init__(self):
        # Binary Cross-Entropy loss function
        def bce(y_pred, y_true):
            # Clipping the values to avoid log(0) which can cause NaNs
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
            return -np.mean((y_true * np.log(y_pred)) + ((1 - y_true) * np.log(1 - y_pred)))

        # Derivative of the Binary Cross-Entropy loss function
        def bce_derivative(y_pred, y_true):
            # Clipping for stability in derivative computation
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
            return (y_pred - y_true) / (y_pred * (1 - y_pred))  # Correct derivative expression

        super().__init__(bce, bce_derivative)

In [131]:
class SparseCategoricalCE(Loss):
    def __init__(self):
        def sce(logits, true_labels):
            # Numerically stable softmax computation
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Shift logits for numerical stability
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)  # Softmax
            true_class_probs = probabilities[np.arange(len(true_labels)), true_labels]  # Select true class probabilities
            log_loss = -np.log(true_class_probs + 1e-15)  # Compute log loss, adding a small constant for stability
            return np.mean(log_loss)  # Mean log loss over the batch

        def sce_derivative(logits, true_labels):
            # Softmax derivative (gradient of cross-entropy)
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Stability trick
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)  # Softmax probabilities
            batch_size = logits.shape[0]

            # Subtract 1 from the probability of the true class
            probabilities[np.arange(batch_size), true_labels] -= 1

            # Gradient of the loss with respect to logits (average over the batch)
            gradient = probabilities / batch_size
            return gradient

        super().__init__(sce, sce_derivative)

In [132]:
class CategoricalCrossEntropyLoss(Loss):
    def __init__(self):
        def cel(y_predSM, y_true):
            # Clip predictions for numerical stability to avoid log(0)
            y_predSM = np.clip(y_predSM, 1e-9, 1 - 1e-9)
            # Compute categorical cross-entropy
            return -np.mean(np.sum(y_true * np.log(y_predSM), axis=1))

        def cel_derivative(y_predSM, y_true):
            # The derivative of categorical cross-entropy with respect to softmax output
            return y_predSM - y_true

        super().__init__(cel, cel_derivative)

In [133]:
class Model:
    def train(self, images, labels, layers, hyperpara):
        for key, value in hyperpara.items():
            setattr(self, key, value)
        self.loss_fn = CategoricalCrossEntropyLoss()
        self.X = images
        self.y = labels
        self.layers = layers
        self.softmax = Softmax(self.neurons[-1], self.softmax_neurons)

        for epoch in range(self.epochs):
            num_samples = self.X.shape[0]
            epoch_loss = 0
            for i in range(0, num_samples, self.batch_size):
                if (i + self.batch_size == num_samples):
                    X_batch = self.X[i:num_samples]
                    y_batch = self.y[i:num_samples]
                else:
                    X_batch = self.X[i:i + self.batch_size]
                    y_batch = self.y[i:i + self.batch_size]

                ActiPredicted = self.forward(X_batch)
                pred_y = self.softmax.forward(ActiPredicted)

                loss = self.loss_fn.forward(pred_y, y_batch)
                epoch_loss += loss

                # Corrected line: Call backward with no additional arguments
                loss_gradient = self.loss_fn.backward()

                # Backward pass
                loss_gradient = self.softmax.backward(loss_gradient, self.learning_rate, self.momentum_coeff)
                self.backward(loss_gradient, self.learning_rate, self.momentum_coeff)

            print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / (num_samples / self.batch_size):.4f}")

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, loss_gradient, learning_rate, momentum_coeff):
        for layer in reversed(self.layers):
            loss_gradient = layer.backward(loss_gradient, learning_rate, momentum_coeff)  # Only pass loss_gradient here

    def predict(self, X):
        logits = self.forward(X)
        probabilities = self.softmax.forward(logits)
        return np.argmax(probabilities, axis=1)

    def evaluate(self, X, y):
        predictions = self.predict(X)
        true_labels = np.argmax(y, axis=1)
        accuracy = np.mean(predictions == true_labels)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        return accuracy

In [134]:
hyperparameters={
    "hiddenLayers":3,
    "learning_rate":0.005,
    "batch_size":32,
    "epochs":150,
    "ActivationFns":None,
    "neurons":None,
    "softmax_neurons":labels.shape[1],
    "momentum_coeff":0.9

}

# change
hyperparameters["neurons"]=[32,32,16]
hyperparameters["ActivationFns"]=[Relu(),Relu(),Relu()]
layers=[]
inputsize=images.shape[1]
for i in range(hyperparameters["hiddenLayers"]):
    outputsize=hyperparameters["neurons"][i]
    layers.extend([Dense(inputsize,outputsize),hyperparameters["ActivationFns"][i]])
    inputsize=outputsize


model=Model()
model.train(images,labels,layers,hyperparameters)

Epoch 1, Average Loss: 2.5457
Epoch 2, Average Loss: 2.3994
Epoch 3, Average Loss: 2.2114
Epoch 4, Average Loss: 2.1405
Epoch 5, Average Loss: 2.0850
Epoch 6, Average Loss: 2.0160
Epoch 7, Average Loss: 1.9242
Epoch 8, Average Loss: 1.8957
Epoch 9, Average Loss: 1.7999
Epoch 10, Average Loss: 1.7685
Epoch 11, Average Loss: 1.7142
Epoch 12, Average Loss: 1.6636
Epoch 13, Average Loss: 1.6725
Epoch 14, Average Loss: 1.6977
Epoch 15, Average Loss: 1.5928
Epoch 16, Average Loss: 1.5176
Epoch 17, Average Loss: 1.4187
Epoch 18, Average Loss: 1.3517
Epoch 19, Average Loss: 1.2315
Epoch 20, Average Loss: 1.1420
Epoch 21, Average Loss: 1.0624
Epoch 22, Average Loss: 1.0430
Epoch 23, Average Loss: 1.0958
Epoch 24, Average Loss: 1.0561
Epoch 25, Average Loss: 1.0135
Epoch 26, Average Loss: 0.8717
Epoch 27, Average Loss: 0.8097
Epoch 28, Average Loss: 0.7324
Epoch 29, Average Loss: 0.7448
Epoch 30, Average Loss: 0.7016
Epoch 31, Average Loss: 0.6193
Epoch 32, Average Loss: 0.5240
Epoch 33, Average

In [135]:
print(f"accuary:{model.evaluate(test_images,test_labels)}")

Accuracy: 73.39%
accuary:0.7339055793991416
